## 1. Installing required libraries

In [ ]:
!pip install pulp

## 2. Importing required libraries

In [2]:
from pulp import *

## 3.Defining the sources, distribution centers, and markets

In [3]:
sources = ['N', 'S']
distribution_centers = ['DC_A', 'DC_B'] 
markets = ['M1', 'M2', 'M3']

## 4.Defining the transportation costs

In [4]:
transportation_costs = {
    ('N', 'DC_A'): 44,
    ('N', 'DC_B'): 50,
    ('S', 'DC_A'): 54,
    ('S', 'DC_B'): 52,
    ('DC_A', 'M1'): 40,
    ('DC_A', 'M2'): 59,
    ('DC_A', 'M3'): 76,
    ('DC_B', 'M1'): 73,
    ('DC_B', 'M2'): 47,
    ('DC_B', 'M3'): 79
}

The keys of the dictionary represent the origin and destination pairs (like Production to distribution center and Distribution Center to Markets). The values represent the corresponding transportation cost.


## 5. Defining the capacities of the sources

In [5]:
capacities = {
    'N': 600,
    'S': 500
}

## 6.Defining the demands of the markets

In [6]:
demands = {
    'M1': 300,
    'M2': 300,
    'M3': 500
}

## 7.Problem Definition

In [7]:
problem = LpProblem("Steak Transportation Problem", LpMinimize)

C:\Users\RAHUL ELDHO BIJOY\anaconda3\lib\site-packages\pulp\pulp.py:1352: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


Here we creates an instance of a linear programming problem using the PuLP library.LpMinimize is used to indicate the objective of the problem.

## 8. Defining the decision variables

In [8]:

X = LpVariable.dicts("Tons", (sources, distribution_centers, markets), lowBound=0, cat='Continuous')


Here we are defining the decision variables for the linear programming problem. We are creating in dictionary of variables which helps to accomodate multiple decision variables.The lowBound=0 sets the lower bound of the decision variables to 0, meaning that the amount of tons shipped cannot be negative.
The variable cat='Continuous': It specifies the category of the decision variables as continuous, meaning they can take on any real value within the given bounds.

## 8.Defining the objective function

In [9]:
problem += lpSum(transportation_costs[source, center] * X[source][center][market]
                 for source in sources
                 for center in distribution_centers
                 for market in markets)

Here we use lpSum  to calculates the sum of a series of terms. By summing up the costs for all combinations (sources, distribution centers, and markets), the objective function captures the total cost of transportation in the problem. The goal of the optimization is to minimize this total cost. ( We can also use in normal sum method)

## 9.Finding the constraints  in source and demand

In [10]:

for source in sources:
    problem += lpSum(X[source][center][market]
                     for center in distribution_centers
                     for market in markets) <= capacities[source]
    


Here I am trying to ensure that the amount of steak shipped from each source (production plant) to the distribution centers and markets does not exceed the capacity of that source. If the sum of amount of steaks(productionplants,distributioncentres and Markets) exceeds the capacity, it violates the constraint, and the problem will consider it as an infeasible solution.

In [11]:
for market in markets:
    problem += lpSum(X[source][center][market]
                     for source in sources
                     for center in distribution_centers) >= demands[market]

Here I am trying to ensure that the demand of each market is satisfied by the total amount of steak shipped from all production plants  and distribution centers.

In [12]:
problem.solve()
print("Optimal solution:")
for source in sources:
    for center in distribution_centers:
        for market in markets:
            print(f"Tons shipped from {source} to {center} and then to {market}: {X[source][center][market].varValue}")

# Print the total cost
print(f"Total cost: {value(problem.objective)} euros")

Optimal solution:
Tons shipped from N to DC_A and then to M1: 0.0
Tons shipped from N to DC_A and then to M2: 100.0
Tons shipped from N to DC_A and then to M3: 500.0
Tons shipped from N to DC_B and then to M1: 0.0
Tons shipped from N to DC_B and then to M2: 0.0
Tons shipped from N to DC_B and then to M3: 0.0
Tons shipped from S to DC_A and then to M1: 0.0
Tons shipped from S to DC_A and then to M2: 0.0
Tons shipped from S to DC_A and then to M3: 0.0
Tons shipped from S to DC_B and then to M1: 300.0
Tons shipped from S to DC_B and then to M2: 200.0
Tons shipped from S to DC_B and then to M3: 0.0
Total cost: 52400.0 euros


## 10. Calculating the tons of steak from DC A and DC B

here we got the  optimal transportation plan by showing how much steak should be shipped from each production plants N,S to each distribution center DC A. DC B and then to each market (M1,M2M3) to minimize the total cost.

In [13]:

tons_through_DC_A = 0
tons_through_DC_B = 0
for source in sources:
    for center in distribution_centers:
        for market in markets:
            if center == 'DC_A':
                tons_through_DC_A += X[source][center][market].varValue
            elif center == 'DC_B':
                tons_through_DC_B += X[source][center][market].varValue

## 11 Calculating the weekly costs

In [14]:
weekly_cost_DC_A = 0
weekly_cost_DC_B = 0

for source in sources:
    for center in distribution_centers:
        for market in markets:
            if center == 'DC_A':
                weekly_cost_DC_A += transportation_costs[(source, center)] * X[source][center][market].varValue
            elif center == 'DC_B':
                weekly_cost_DC_B += transportation_costs[(source, center)] * X[source][center][market].varValue


## 12.Final Calculated values

In [15]:
print("Tons shipped through DC A:", tons_through_DC_A)
print("Tons shipped through DC B:", tons_through_DC_B)
print("Weekly cost through DC A:", weekly_cost_DC_A, "euros")
print("Weekly cost through DC B:", weekly_cost_DC_B, "euros")

Tons shipped through DC A: 600.0
Tons shipped through DC B: 500.0
Weekly cost through DC A: 26400.0 euros
Weekly cost through DC B: 26000.0 euros


reference: https://machinelearninggeek.com/solving-transportation-problem-using-linear-programming-in-python/